# Testowanie działania zapisanego modelu

## Utworzenie ramki danych

In [ ]:
from typing import Dict, List, Any, Self, Optional

import numpy as np
# Konkatenacja powstałych datasetów csv w 1, wraz z transformacją kolumn math, bio, code w jedną - category,
# której jedyne wartości 0, 1, 2 odnoszą się odpowiednio do kategorii math, bio, code.

import pandas as pd
import glob
import sys
import os

from matplotlib.pyplot import title
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

if project_root not in sys.path:
    sys.path.append(project_root)

csv_files = glob.glob('../datasets_preprocessing/csv_question_files/*.csv')

dfs = []
for file in csv_files:
    tmpdf = pd.read_csv(file)
    sample = tmpdf.sample(n=4000)
    dfs.append(sample)

df = pd.concat(dfs, ignore_index=True)

def transform_categories(math, bio, code):
    if math == 1:
        return 0
    elif bio == 1:
        return 1
    elif code == 1:
        return 2
    return 2

df['category'] = df.apply(lambda row: transform_categories(row['math'], row['bio'], row['code']), axis=1)

# Usunięcie kolumn math, bio, code z df — są zastąpione category
df.drop(['math', 'bio', 'code'], axis=1, inplace=True)

## Funkcja do załadowania wybranego zbioru testowego

In [ ]:
from train.reporting.model_interface import ModelInterface
from train.reporting.svm_model_wrapper import SVMModelWrapper
from typing import Tuple
import pickle

def import_model_and_test_set(path: str) -> Tuple[ModelInterface, pd.DataFrame]:
    with open (path + "/model.pkl", 'rb') as f:
        model = pickle.load(f)

    test_set = pd.read_csv(path + "/test_set.csv", index_col=0)

    return model, test_set

model, _ = import_model_and_test_set("saved_model/1model-saved")

In [ ]:
test_set_number = 0 # 1 lub 2

In [ ]:
test_df = pd.read_csv(f"../datasets_preprocessing/datasets/test_all_models/test_{test_set_number}.csv", index_col=0)

test_df.drop_duplicates(subset=["question"], keep="first")
test_df

## Dodanie kolumny określającej kategorię pytania

In [ ]:
test_df['real_class'] = (test_df['math'] * 0 + test_df['bio'] * 1 + test_df['code'] * 2)

## Badanie predykcji modelu dla każdej kategorii i zapisanie ich w ramce danych

In [ ]:
proba_arr = model.predict_proba(test_df["question"])

test_df["math_preds"] = proba_arr[:, 0]
test_df["bio_preds"] = proba_arr[:, 1]
test_df["code_preds"] = proba_arr[:, 2]

In [ ]:
test_df

## Zapisanie przewidzianych klas w ramce danych (-1, gdy maksymalne prawdopodobieństwo nie przekracza 0.5)

In [ ]:
import numpy as np

cols = ['math_preds', 'bio_preds', 'code_preds']

max_vals = test_df[cols].max(axis=1)
max_names = test_df[cols].idxmax(axis=1)

class_mapping = {'math_preds': 0, 'bio_preds': 1, 'code_preds': 2}
predicted_classes = max_names.map(class_mapping)

test_df['predicted_class'] = np.where(max_vals > 0.5, predicted_classes, -1)
test_df

## Efektywność modelu

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(test_df['real_class'], test_df['predicted_class'])

## Przykładowe pytania, dla których model niepoprawnie przewidział kategorię

In [ ]:
wrong_questions_sample = test_df[test_df['real_class'] != test_df['predicted_class']].sample(n=10)

wrong_questions_sample

In [ ]:
for question in wrong_questions_sample['question']:
    print(question)